In [1]:
a = "abba"
b = "ccd"
len(a)

4

In [2]:
import numpy as np

In [3]:
#Generator of non equivalent word on a certain alphabet and with certain size.
#the parametter lastLetter is only usefull for the recursion. 
def allWord(alphabet, size, lastLetter=0):
    if size == 0:
        yield ""
    else:
        for i in allWord(alphabet, size - 1, lastLetter):
            for j in range(lastLetter):
                yield alphabet[j] + i
        if lastLetter < len(alphabet):
            for i in allWord(alphabet, size-1, lastLetter + 1):
                yield alphabet[lastLetter] + i

In [4]:
for i in allWord("ab", 4, 0):
    print(i)

aaaa
aaab
aaba
aabb
abaa
abba
abab
abbb


In [5]:
def calcul_r(u: str) -> np.array:
    """Calcule du r-vecteur de u"""
    n = len(u)
    # we initialize the r vector
    r = np.zeros(n, dtype=int)

    for i in range(n):  # calcul de r(u[:i], u[i])
        # for each letter c of the word u, and a prefix u' s.t. u=u'cu''
        # we look at w, first suffix of u' starting with the letter c

        # indice de la dernière occurence de u[i] dans u[:i]
        start_of_w = u.rfind(u[i], 0, i)

        if start_of_w == -1:
            r[i] = 0
        else:
            r[i] = 1 + np.min(r[start_of_w:i])
    return r

In [6]:
def period(word, n):
    """
    Return the (period, transient)
    """
    # Create r-vector
    r_vector = calcul_r(word * n)
    r_chunks = []
    
    lword = len(word)

    # Cut the r-vector into chunks of length of word
    for i in range(n):
        r_chunks.append(r_vector[(lword*i):(lword*(i+1))])

    # We iterate over different periods
    for p in range(1, n):
        previous = None # last difference vector
        same = False # tell if the previous difference vector is the same as the current one

        # We iterate over the chunks, starting from the last part
        for i in range(n-1, p - 2, -1):
            diff = r_chunks[i] - r_chunks[i - p]

            # First case, we don't have anything to compare to
            if previous is None:
                previous = diff

            # Other cases
            else:
                if not(np.array_equal(previous, diff)) and not(same):
                    break
                elif not(np.array_equal(previous, diff)) and same:
                    return (p, i-p + 1)
                else:
                    same = True

In [7]:
period("abaccdd", 10)

(1, 3)

In [8]:
for i in allWord("a", 12, 0):
    print(i)

aaaaaaaaaaaa


In [9]:
for i in allWord("abcde", 6, 0):
    p, k = period(i, 10)
    print(f"la peride du mot {i} est {p} et sont transiant est de taille {k}")

la peride du mot aaaaaa est 1 et sont transiant est de taille 0
la peride du mot aaaaab est 1 et sont transiant est de taille 0
la peride du mot aaaaba est 1 et sont transiant est de taille 1
la peride du mot aaaabb est 1 et sont transiant est de taille 1
la peride du mot aaaabc est 1 et sont transiant est de taille 0
la peride du mot aaabaa est 1 et sont transiant est de taille 1
la peride du mot aaabba est 1 et sont transiant est de taille 0
la peride du mot aaabab est 1 et sont transiant est de taille 0
la peride du mot aaabbb est 1 et sont transiant est de taille 1
la peride du mot aaabac est 1 et sont transiant est de taille 0
la peride du mot aaabbc est 1 et sont transiant est de taille 0
la peride du mot aaabca est 1 et sont transiant est de taille 1
la peride du mot aaabcb est 1 et sont transiant est de taille 1
la peride du mot aaabcc est 1 et sont transiant est de taille 1
la peride du mot aaabcd est 1 et sont transiant est de taille 0
la peride du mot aabaaa est 1 et sont tr

In [10]:
#this fonction made it's test on all word of a given size and a given alphabet
#the parametter n is for the powaer apply to the word to test their period
#the result is a dictionary which associate the word with a size > Tperiode and the size of their period
def grandePeriode(alphabet, size, n, Tperiode):
    res = {}
    for i in allWord(alphabet, size, 0):
        p, k = period(i, n)
        if p >= Tperiode:
            res[i] = p
    return res

In [36]:
grandePeriode("a", 12, 10, 3)

{}

In [11]:
#this fonction is searching on all word of a given size and a given alphabet
#the one with the largest period and the ones with the largest transient
#the result is doublet, the first element is a doublet containing the word with the largest period and the size of it's period.
#The second element is a doublet containing the word with the largest transient and the size of it's transient
def maxPeriodAndTranscient(alphabet, size, n):
    resP = ("",0)
    resT = ("", 0)
    for i in allWord(alphabet, size, 0):
        p, k = period(i, n)
        if p > resP[1]:
            resP = (i,p)
        if k > resT[1]:
            resT = (i, k)
            
    return resP, resT

In [52]:
maxPeriodAndTranscient("abcdef", 12, 10)

(('aabbccddeeff', 5), ('aabaccddeeff', 5))

In [12]:
def conjugate(word):
    for idx in range(len(word)):
        yield word[idx:] + word[:idx]

In [14]:
for i in conjugate("aabbcc"):
    print(i)

aabbcc
abbcca
bbccaa
bccaab
ccaabb
caabbc


In [15]:
def slope(word):
    """
    Calculate the slope of the word
    """
    p, k = period(word, 10) 
    r_vector = calcul_r(word * 10)
    n = len(word)
    # We look at the end of the word 
    diff = r_vector[n * 9: n * 10] - r_vector[n * (9 - p): n * (10 - p)]

    return diff[0]

In [21]:
def conjugateSlopeAndPeriod(alphabet, size, n):
    for i in allWord(alphabet, size):
        P = period(i, n)[0]
        S = slope(i)
        for j in conjugate(i):
            Pj = period(j, n)[0]
            if Pj != P:
                print(f"le mot {i} n'a pas la meme periode que son conjugué {j} respectivement taille {P}, {Pj}")
            Sj = slope(j)
            if Sj != S:
                print(f"le mot {i} n'a pas la meme pente que son conjugué {j} respectivement taille {S}, {Sj}")

In [ ]:
conjugateSlopeAndPeriod("abcde", 10, 10)